In [2]:
import json
import pandas as pd
import re
from datetime import datetime, timedelta
from typing import List, Dict, Tuple
from pydantic import BaseModel, Field, validator
from ollama import chat
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(api_key=os.getenv('OAI_API_KEY'))

In [3]:
progress_notes = pd.read_csv('results/datasets/combined_hospital_progress_medications_analysis.csv')

In [12]:


for idx, row in progress_notes.iloc[1:2].iterrows():
    hadm_id = row['HADM_ID']
    progress_note = row['Progress_note']
    medication = row['medication']
    with open(f'results/notes/a_p_{hadm_id}.json', 'r') as f:
        assessment_plan = json.load(f)
    
    cot_prompt = f"""
        You are provided with the following:
        1. A patient progress note
        2. The Assessment and Plan section of the note
        3. A specific medication that was started on the same day as the note
        Your task is to reason step-by-step like the treating physician that lead to the final conclusion that this medication needs to be started.

        Guidelines for Reasoning:

        1. Use a clinical reasoning tone (e.g. Alright, let’s break this down...Okay, let’s reconsider...Oh, wait a second, maybe we're missing something simpler...But hang on, what if...). As you are reasoning to the conclusion (the medication), the medication itself should not occur in the reasoning process.

        2.  Summarize the clinical course and intergrate clinical data (chief complaint, past medical history, recent events, vital signs, lab results, imaging findings, current medications, etc. MENTION SPECIFIC CLINICAL DATA!) in your reasoning; remember, imaging findings are usually important. 

        3. Review the Assessment and Plan section.
            - It may explicitly state the reasoning for the medication.
            - However, do not assume the reasoning is correct, assess it critically.
            - DO NOT CITE ANYTHING FROM THE ASSESSMENT AND PLAN SECTION, but use it as if you come up with the reasoning yourself.
        
        4. Use this as the final sentence: Therefore, due to (summary of reasons), XXX (medication) was started today.

        ### Input Format:
        Progress Note:
        {progress_note}

        Assessment and Plan:
        {assessment_plan}

        Medication Started:
        {medication}

        ### Output Format:
        Reasoning (Step-by-step clinical reasoning process):
        {{Your detailed reasoning trace}}

        """

    response = client.chat.completions.create(
        model="o3-mini-2025-01-31",
        messages=[{"role": "user", "content": cot_prompt}]
    )
    
    # Print or save the response
    print(response.choices[0].message.content)


Reasoning (Step-by-step clinical reasoning process):

Alright, let’s break this down. We have a 43‐year‐old gentleman with a history of recurrent diverticulitis who has undergone several significant surgical interventions. His course began with a laparoscopic sigmoid colectomy for recurrent episodes and then evolved into more extensive procedures postoperatively due to complications—specifically, an anastomotic leak as evidenced by a CT scan demonstrating extraluminal contrast extravasation and the subsequent need for drainage and additional surgical management.

On top of the abdominal complications, the patient’s systemic response has been notable. His recent vital signs have shown fevers with Tmax values reaching about 38.7–38.8°C and episodes of tachycardia with heart rates in the 120–140 range. His laboratory findings reinforce the picture of significant inflammation or infection with markedly elevated white blood cell counts (up to 41.0 K/uL) and a drop in hematologic indices tha